In [1]:
# %load imports
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium.webdriver.chrome.options import Options as ChromeOptions
from datetime import datetime
CHROME_DRIVER = './chromedriver'
FIREFOX_DRIVER = './geckodriver'
import time
import re
import requests
import json
import random
import copy
import pprint


def setup_driver(headless=True, driver_type=CHROME_DRIVER) -> webdriver:
    if 'chrome' in driver_type.lower():
        options = ChromeOptions()
        options.add_argument('--ignore-certificate-errors')
        options.add_argument('--ignore-ssl-errors')
        if headless:
            options.add_argument('-headless')
        driver = webdriver.Chrome(executable_path=driver_type, chrome_options=options)
    else:
        options = FirefoxOptions()
        if headless:
            options.add_argument('-headless')
        driver = webdriver.Firefox(executable_path=driver_type, firefox_options=options)
    return driver


In [2]:
def get_soup(link):
    soupd = {}
    #basic function to parse the soup
    driver = setup_driver()
    driver.get(link)
    time.sleep(8)
    soup = BeautifulSoup(driver.page_source, "html.parser")
    executing_query = "document.getElementsByClassName('popover-x-button-close icl-CloseButton')[0].click()"
    #in case a pop up appears, it closes it
    if soup.find('div',{'id':'popover-form-container'}):
        driver.execute_script(executing_query)
    soupd[driver.current_url] = soup
    driver.quit()
    return soup


def parse_all_pages(page):
    #from the first page, it gets the links to all other pages..
    page_links = []
    blocks = page.find('ul',{'class':'pagination-list'}).find_all('li')
    for block in blocks:
        if block.find('a', href = True):
            incomplete_link = block.find('a')['href']
            page_links.append('https://www.indeed.com'+incomplete_link)
            
    pages = []
    #stores the soup of all the pages
    pages.append(page)
    #appends the soup for the first page before getting into the loop
    for link in set(page_links):
        pages.append(get_soup(link))
        #loops with all the links to other pages and stores their soups..
    return pages

def get_all_soups(pages):
    soups = []
    #stores the soups to each job postings, the soup from their specific 
    links_to_postings = []
    #stores all the links to the specific job postings
    for page in pages:
        method_1 = page.find('div',{'class':'mosaic mosaic-provider-jobcards mosaic-provider-hydrated'})
        method_2 = page.find('div',{'class':'jobsearch-SerpJobCard unifiedRow row result clickcard'})
        #this takes care of the two different classes used in the html pages
        if method_1:
            container = method_1
            boxes = container.find_all('a', href = True)
        elif method_2:
            container = method_2
            boxes = container.find_all('a', href = True)
        else:
            return "we don't have the job postings"
            # quits the method if the links are not found.
            
        if boxes: #proceeds only if links to different postings exists
            for box in boxes:
                first = box['href']
                if '/rc/clk?jk' in first:
                    link = first.replace('/rc/clk?','https://www.indeed.com/viewjob?')
                    links_to_postings.append(link)
                    #append all the links to the list
    for link in set(links_to_postings):
        soups.append(get_soup(link))
        #get soups for unique job links parsed from different websites
    print(f'we got {len(soups)} job postings')
    return soups

In [3]:
class job_posting(object):
    def __init__(self):
        self.job_title = None
        self.company = None
        self.original_link = None
        self.is_full_time = None
        self.is_remote = None
        self.job_location = None
        self.requirements = None
        self.languages_used = None
       

In [4]:
page = get_soup('https://www.indeed.com/jobs?q=%22Software+Engineer%22&l=')

<ipython-input-1-42be7db9a6ee>:26: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path=driver_type, chrome_options=options)


https://www.indeed.com/jobs?q=%22Software+Engineer%22


In [5]:
pages = parse_all_pages(page)

<ipython-input-1-42be7db9a6ee>:26: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path=driver_type, chrome_options=options)


https://www.indeed.com/jobs?q=%22Software+Engineer%22&start=40
https://www.indeed.com/jobs?q=%22Software+Engineer%22&start=30
https://www.indeed.com/jobs?q=%22Software+Engineer%22&start=10
https://www.indeed.com/jobs?q=%22Software+Engineer%22&start=20


In [6]:
soups = get_all_soups(pages)

<ipython-input-1-42be7db9a6ee>:26: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path=driver_type, chrome_options=options)


https://www.indeed.com/viewjob?jk=c1e5bd7a50cac627&fccid=5309ddd82b852332&vjs=3
https://www.indeed.com/viewjob?jk=4e46d4925e6c6328&fccid=3e3eaa6aedf690a3&vjs=3
https://www.indeed.com/viewjob?jk=16e97e1fd4ed4a4b&fccid=e32d933c26e873c8&vjs=3
https://www.indeed.com/viewjob?jk=835c0973db3a626b&fccid=6bcbaf35af26e1a6&vjs=3
https://www.indeed.com/viewjob?jk=76ddcf435e07a36e&fccid=c594c442f5397e7b&vjs=3
https://www.indeed.com/viewjob?jk=6797ab67821bf99e&fccid=f490aa2d3c80bc75&vjs=3
https://www.indeed.com/viewjob?jk=7bcc0e8542a54815&fccid=bee6abba453010ff&vjs=3
https://www.indeed.com/viewjob?jk=c86732b141b38515&fccid=c007936ceb766fe5&vjs=3
https://www.indeed.com/viewjob?jk=a359dd1de8ba0947&fccid=cd6c9f98394f9865&vjs=3
https://www.indeed.com/viewjob?jk=3db8678198e5f5d4&fccid=25b5166547bbf543&vjs=3
https://www.indeed.com/viewjob?jk=c301ab0af8d9b051&fccid=68ec8a8ee6cf5ac6&vjs=3
https://www.indeed.com/viewjob?jk=1c1cca2129ef17d6&fccid=9784ae78e9834539&vjs=3
https://www.indeed.com/viewjob?jk=6c9db1

In [19]:
def parse_job_title(soup):
    title = 'N/A'
    title_text = soup.find('h1',{'class':'icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title'})
    if title_text:
        title = title_text.text.title()
    return title    
    
    
def parse_link(soup):
    link = 'N/A'
    link_text = soup.find('div',{'id':'originalJobLinkContainer'})
    if link_text:
        link = link_text.find('a')['href']
    return link
        
        
def parse_company_name(soup):
    name = 'unknown'
    text = soup.find('div',{'class':'jobsearch-InlineCompanyRating icl-u-xs-mt--xs jobsearch-DesktopStickyContainer-companyrating'})
    if text:
        name = re.sub('[0-9]','',text.text)
    final = name.replace('reviews','').replace(',','').title()
    return final


def parse_working_time(soup):
    soup_text = soup.text.lower()
    if 'full time' in soup_text or 'full-time' in soup_text:
        full_time = True
    elif 'part time' in soup_text or 'part-time' in soup_text:
        full_time = False
    else:
        full_time = 'N/A'
    return full_time
    
    
def parse_job_location(soup):
    is_remote = 'N/A'
    final = 'N/A'
    entire_text = soup.find('div',{'class':'icl-u-xs-mt--xs icl-u-textColor--secondary jobsearch-JobInfoHeader-subtitle jobsearch-DesktopStickyContainer-subtitle'})
    if entire_text:
        company_name = soup.find('div',{'class':'jobsearch-InlineCompanyRating icl-u-xs-mt--xs jobsearch-DesktopStickyContainer-companyrating'}).text
        final = entire_text.text.replace(company_name, '')
    if 'remote' in final.lower():
        final = final.lower().split('remote')[0].title()
        is_remote = True
    if final == '':
        final = 'Remote'
    return final , is_remote
    
    
    

In [22]:
for soup in soups:
    job = job_posting()
    job.job_title = parse_job_title(soup)
    job.original_link = parse_link(soup)
    job.company = parse_company_name(soup)
    job.is_full_time = parse_working_time(soup)
    job.job_location, job.is_remote = parse_job_location(soup)
    
    job.requirements = None
    job.languages_used = None
    